In [1]:
# Importing required packages
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from scipy.optimize import fsolve, fminbound, fmin_bfgs

In [2]:
# Quasi-Newton unconstained optimization 
def quasi_newton_opt(x0, func, func_grad, bounds, max_iter=50, full_output=False):
    # Function to perform a quasi-Newton optimization
    # for an unconstrained problem with BFGS Hessian update

    # x0: Inital point
    # func: objective function
    # func_grad: Gradient of function
    # bounds: design variable bounds
    # max_iter: maximum number of iterations 
    # full_output: option to enable full output

    B0 = np.identity(len(x0))    # initializing hessian
    iter_no = 0
    while True:
        delta_f0 = func_grad(x0, func)    # calculating gradient
        sq = np.dot(-B0, delta_f0)  # calculating search direction
        sq = sq / np.sqrt(np.sum(sq**2))    # normalizing search direction

        alpha_max = [0]
        for i in range(len(sq)):
            if sq[i] > 0:
                alpha_max = max(alpha_max, (bounds[i][1] - x0[i])/sq[i])
            else:
                alpha_max = max(alpha_max, (bounds[i][0] - x0[i])/sq[i])

        alpha = fminbound(lambda alpha: func(x0 + alpha*sq), 0, alpha_max[0])  # line search along sq
        x_new = x0 + alpha * sq # new point
        delta_f_new = func_grad(x_new, func)  # gradient at new point 
        delta_x = x_new - x0    # change in x
        delta_delta = delta_f_new - delta_f0    # change in gradient

        # Checking for stationary point or max iterations:
        if max(abs(delta_f_new)) < 1E-06:
            exit_condition = 'Gradient less than tolarence'
            break
        elif max(abs(delta_x)) < 1E-06:
            exit_condition = 'Design point update less than tolarence'
            break
        elif iter_no >= max_iter:
            exit_condition = 'Max iteration reached'
            break

        
        delta_B = BFGS_update(B0, delta_x, delta_delta) # BFGS hessian update

        # Updating values for next iteration
        B_new = B0 + delta_B
        B0 = B_new
        x0 = x_new
        delta_f0 = delta_f_new
        iter_no = iter_no + 1

    # Output
    if full_output:
        return [x_new, delta_f_new, B_new, iter_no, exit_condition]
    return x_new

In [3]:
# Constrained augmented lagrangian optimization
def augmented_lagrangian(x0, func, func_grad, con, bounds, rho0=1, max_iter=50, full_output=False):
    # Function to perform a augmented Lagrangian optimization
    # for an unconstrained problem with quasi-Newton unconstrained optimization

    # x0: Inital point
    # func: objective function
    # func_grad: Gradient of function
    # con: constraint functions
    # bounds: design variable bounds
    # rho: penalty value
    # max_iter: maximum number of iterations 
    # full_output: option to enable full output

    g, h = con(x0)  # constraints at inital point
    m0 = np.zeros((len(g), 1))  # initializing mu
    l0 = np.zeros((len(h), 1))  # initializing lambda
    m = np.array(m0)    # mu array
    l = np.array(l0)    # lambda array

    # Augmented Lagrangian loop
    iter_no = 0 
    while True:
        x = quasi_newton_opt(x0, lambda x: calc_aug_lagrangian(x, func, con, m0, l0, rho0), func_grad, bounds)  # performing quasi-Newton opt. on augmented Lagrangian

        g, h = con(x)   # constraint value at new design point
        m = m0 + rho0*g # updating mu
        for i in range(len(m0)):
            m[i] = max(0, m[i]) 
        l = l0 + rho0*h # updating lambda

        delta_L = func_grad(x, lambda x: calc_lagrangian(x, func, con, m, l))   # gradient of Lagrangian at new design point
        delta_x = x - x0    # change in design point

        # Checking for stationary point or max iterations:
        if max(abs(delta_L)) < 1E-06:
            exit_condition = 'Gradient less than tolarence'
            break
        elif max(abs(delta_x)) < 1E-06:
            exit_condition = 'Design point update less than tolarence'
            break
        elif iter_no >= max_iter:
            exit_condition = 'Max iteration reached'
            break

        # Updating variables for next iteration
        x0 = x
        m0 = m
        l0 = l
        iter_no = iter_no + 1

    # Output    
    if full_output:
        return (x, delta_L, m, l, g, h, iter_no, exit_condition)
    return x

def calc_lagrangian(x, func, con, m, l):
    # Function to calculate Lagrangian 

    # x: Evaluation point
    # func: objective function
    # con: constraint function
    # m: mu
    # l: lambda

    g, h = con(x)   # calculating constraints at evaluation point

    # Calculating Lagrangian
    if h.size and g.size:   # both h and g present
        return func(x) + np.dot(m.transpose(), g) + np.dot(l.transpose(), h)
    elif h.size:    # only h present
        return func(x) + np.dot(l.transpose(), h)
    else:   # only g present
        return func(x) + np.dot(m.transpose(), g)



def calc_aug_lagrangian(x, func, con, m, l, rho):
        # Function to calculate Lagrangian 

    # x: Evaluation point
    # func: objective function
    # con: constraint function
    # m: mu
    # l: lambda
    # rho: quadratic penalty

    g, h = con(x)   # calculating constraints at evaluation point

    # Calculating augmented Lagrangian
    if h.size and g.size:   # both h and g present
        return func(x) + np.dot(m.transpose(), g) + np.dot(l.transpose(), h) + 0.5 * rho * ((np.dot(g.transpose(), g))**2 + (np.dot(h.transpose(), h))**2)
    elif h.size:    # only h present
        return func(x) + np.dot(l.transpose(), h) + 0.5 * rho * ((np.dot(h.transpose(), h))**2)
    else:   # only g present
        return func(x) + np.dot(m.transpose(), g) + 0.5 * rho * ((np.dot(g.transpose(), g))**2)


In [4]:
def func(x):
    # Objective function definition

    # x: Evaluation point

    f = 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2 
    return f

def con(x):
    # Constraint function definition

    # x: Evaluation point

    g = np.array([x[0]**2 + x[1]**2 - 0.5])
    h = np.array([])
    return (g, h)


def penalty(x, p=100):
    # Penalty function to convert constrained to unconstrained problem 
    
    # x: Evaluation point
    # p: quadratic penalty

    con_sum = 0 # initializing total penalty 
    g, h = con(x)   # calculating constraints at evaluation point

    # Calculating penalty
    for gi in g:
        con_sum = con_sum + max(0, gi)
    for hi in h:
        con_sum = con_sum + max(0, hi)
    return func(x) + p * (con_sum)**2


def func_grad(x, func):
    # Function to calculate gradient 
    # of objective function using forward FD

    # x: Evaluation point

    step = 1E-8 # step size for FD
    delta_f = np.zeros((len(x), 1)) # initializing gradient
    for i in range(len(x)):
        x_temp = np.array(x, dtype=float)
        x_temp[i] = x[i] + step
        delta_f[i] = (func(x_temp) - func(x)) / step    # calculating gradient using forward FD
    return delta_f



def BFGS_update(B, delta_x, delta_delta):
    # Function to update to hessian
    # using BFGS method

    # B: initial hessian
    # delta_x: change in evaluation point
    # delta_delta: change in gradient
    
    delta_x_T = delta_x.transpose()
    delta_delta_T = delta_delta.transpose()
    delta_B = (1 + (np.dot(np.dot(delta_delta_T, B), delta_delta))/(np.dot(delta_x_T, delta_delta))) * \
    (np.dot(delta_x, delta_x_T)/np.dot(delta_x_T, delta_delta)) - \
        (np.dot(delta_x, np.dot(delta_delta_T, B)) + np.dot(np.dot(delta_delta_T, B).transpose(), delta_x_T)) / \
            (np.dot(delta_x_T, delta_delta))
    
    return delta_B

In [5]:
def scale(x, x_ref):
    # Function to scale design variables to order of 1

    # x: unscaled design variables
    # x_ref: reference motor values

    return x / x_ref

def descale(x_scaled, x_ref):
    # Function to descale design variables

    # x_scaled: scaled design variables
    # x_ref: reference motor values

    return x_scaled * x_ref

def calc_obj_func(x_scaled):
    # Function to calculate the objective function 

    # x: unscaled design variables
    # x_ref: reference motor values

    import constants as c   # importing constants
    x = descale(x_scaled, c.x_ref)  # descaling the design valiables
    P_c, A_t, A_e = x
    t_cyl = (P_c + 10E+05) * c.R_tank * c.f / c.sigma_tank  # thickness of cylindrical tank sections [m]
    t_sph = t_cyl / 2   # thickness of spherical tank sections [m]

    mass_UDMH_tank = 1.2 * (2*t_cyl*np.pi*c.R_tank*c.L_UDMH_tank + 4*t_sph*c.R_tank**2) * c.rho_tank    # mass of UDMH tank [kg]
    mass_N2O4_tank = 1.2 * (2*t_cyl*np.pi*c.R_tank*c.L_N2O4_tank + 4*t_sph*c.R_tank**2) * c.rho_tank    # mass of N2O4 tank [kg]

    mass_flow = c.Gamma * P_c * A_t / np.sqrt(c.R * c.T_c)  # mass flow through the nozzle [kg/s]
    A_c = (mass_flow * c.R * c.T_c) / (0.3 * P_c * np.sqrt(c.gamma * c.R * c.T_c))  # chamber exit area [m2]
    R_c = np.sqrt(A_c / np.pi)  # chamber radius [m]
    k_loads = 1 # correction for high chamber pressure
    V_c = np.pi * R_c**2 * c.L_c    # chamber volume [m3]
    chamber_mass = k_loads * (2/(c.L_c/R_c) + 2) * c.rho/c.sigma * c.f * P_c * V_c  # chamber mass [kg]
    injector_mass = c.rho/c.sigma * c.f * (1.2 * A_c * R_c * np.sqrt(P_c*c.sigma))  # injector plate mass [kg]

    # nozzle_convergent_mass = (rho/sigma) * f * (A_t * (((A_c/A_t)-1)/sind(beta)) * (P_c*R_c));  % convergent nozzle mass using thin shell theory (kg)
    nozzle_divergent_mass = (c.rho/c.sigma) * c.f * (A_t * (((A_e/A_t)-1)/np.sin(c.alpha)) * (P_c*R_c)) # nozzle mass [kg]

    total_mass = chamber_mass + injector_mass + nozzle_divergent_mass + mass_UDMH_tank + mass_N2O4_tank # total motor mass [kg]
    
    return total_mass/c.mass_dry_ref

In [6]:
# Test penalty optimization

x0 = np.array([[0, 0]]).transpose()
bounds = np.array([[-5,-5], [5, 5]], dtype=float).transpose()
[x, delta_f, B, iter_no, exit_condition] = quasi_newton_opt(x0, penalty, func_grad, bounds, max_iter=50, full_output=True)
print(f'Optima: \n{x}\nGradient at optima: \n{delta_f}\nHessian inverse at optima: \n{B}')
print(f'Iterations: {iter_no}')
print(exit_condition)

Optima: 
[[0.6063714 ]
 [0.36631198]]
Gradient at optima: 
[[-0.00017675]
 [-0.00074708]]
Hessian inverse at optima: 
[[0.00172857 0.00036459]
 [0.00036459 0.00332228]]
Iterations: 50
Max iteration reached


In [7]:
# Test penalty optimization with inbuilt unconstrainted optimizer

x0 = np.array([0, 0], dtype=float)
x = fmin_bfgs(penalty, x0)
print(x)
print(con(x))

Optimization terminated successfully.
         Current function value: 0.155482
         Iterations: 15
         Function evaluations: 90
         Gradient evaluations: 30
[0.60637197 0.36631452]
(array([0.0018733]), array([], dtype=float64))


In [8]:
# Test augmented Lagrangian optimization

x0 = np.array([[0, 0]]).transpose()
bounds = np.array([[-5,-5], [5, 5]], dtype=float).transpose()
x, delta_L, m, l, g, h, iter_no, exit_condition = augmented_lagrangian(x0, func, func_grad, con, bounds, rho0=1.1, full_output=True)
print(f'Optima: \n{x}\nGradient of L at optima: \n{delta_L}\nmu: \n{m}\nlambda: \n{l}\ng: \n{g}\nh: \n{h}\n')
print(f'Iterations: {iter_no}')
print(exit_condition)

Optima: 
[[0.60548143]
 [0.3652325 ]]
Gradient of L at optima: 
[[ 1.41553436e-05]
 [-3.35287353e-06]]
mu: 
[[0.37653719]]
lambda: 
[]
g: 
[[2.53620682e-06]]
h: 
[]

Iterations: 6
Design point update less than tolarence
